# Assignment 10.2

> Replace all TODOs with your code. Do not change any other code.

In [2]:
# Do not edit this cell

import csv
import unittest


## Clean code

### Task 1

You are given a function that reads a csv file with temperature measurements (see example below), converts Fahrenheit values to Celsius, calculates and prints some statistics, and writes to another file. It looks a bit messy, let's clean it up!

Example file:
```csv
Temperature (F)
78.5
81.2
75.9
82.1
```

Do the steps below one by one, editing the code in the cell:
1. Naming is so ambiguous and unclear, let's rename variables and function name with proper names.
2. Are these comments really useful?
3. This function does quite a lot, let's divide it in the way that each function does only one thing, and there's one main function that uses others.
4. There seem to be some magic coefficients in the temperature conversion part; let's make them obvious.

If you find any additional improvements, feel free to implement them and leave a comment under your code with an explanation.

In [3]:
# Constants for temperature conversion
FAHRENHEIT_TO_CELSIUS_SCALE = 5 / 9
FAHRENHEIT_TO_CELSIUS_OFFSET = 32


def read_temperatures_from_csv(input_file_path):
    """Reads Fahrenheit temperatures from a CSV and returns a list of temperatures in Celsius."""
    temperatures_celsius = []
    with open(input_file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            temperature_fahrenheit = float(row[0])
            temperature_celsius = (temperature_fahrenheit - FAHRENHEIT_TO_CELSIUS_OFFSET) * FAHRENHEIT_TO_CELSIUS_SCALE
            temperatures_celsius.append(temperature_celsius)
    return temperatures_celsius


def calculate_statistics(temperatures):
    """Calculates and returns the average, minimum, and maximum of a list of temperatures."""
    average_temperature = sum(temperatures) / len(temperatures)
    minimum_temperature = min(temperatures)
    maximum_temperature = max(temperatures)
    return average_temperature, minimum_temperature, maximum_temperature


def write_temperatures_to_csv(temperatures, output_file_path):
    """Writes a list of Celsius temperatures to a CSV file."""
    with open(output_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Temperature (C)'])
        for temperature in temperatures:
            writer.writerow([temperature])


def print_statistics(average_temperature, minimum_temperature, maximum_temperature):
    """Prints the average, minimum, and maximum temperatures."""
    print("Statistics:")
    print(f"Average: {average_temperature:.2f}°C")
    print(f"Minimum: {minimum_temperature:.2f}°C")
    print(f"Maximum: {maximum_temperature:.2f}°C")


def process_temperature_data(input_file_path, output_file_path):
    """Main function to process temperature data."""
    temperatures_celsius = read_temperatures_from_csv(input_file_path)
    average_temperature, minimum_temperature, maximum_temperature = calculate_statistics(temperatures_celsius)
    print_statistics(average_temperature, minimum_temperature, maximum_temperature)
    write_temperatures_to_csv(temperatures_celsius, output_file_path)


### Task 2

How would you write tests for the initial implementation? What exactly would you test in the function?

I hope you see now that once functionality is separated, it's easier to test it in isolation. So, let's write a couple of unit tests for your function and one integration test for your main function.

Hint: you would probably want to mock reading from/writing to file to make the test independent from the environment.

In [6]:
import unittest
from unittest.mock import patch, mock_open

class UnitTestCase(unittest.TestCase):

    @patch("builtins.open", new_callable=mock_open, read_data="Temperature (F)\n78.5\n81.2\n75.9\n82.1")
    def test_read_temperatures_from_csv(self, mock_file):
        # Test that temperatures are correctly read and converted from Fahrenheit to Celsius
        result = read_temperatures_from_csv("fake_file.csv")
        expected = [(temp - 32) * 5 / 9 for temp in [78.5, 81.2, 75.9, 82.1]]

        # Check that each pair of corresponding elements in 'result' and 'expected' are almost equal
        for actual, exp in zip(result, expected):
            self.assertAlmostEqual(actual, exp, places=7)

    def test_calculate_statistics(self):
        # Test statistics calculation
        temperatures_celsius = [25.83, 27.33, 24.39, 27.83]
        average, minimum, maximum = calculate_statistics(temperatures_celsius)
        self.assertAlmostEqual(average, 26.345, places=2)
        self.assertEqual(minimum, 24.39)
        self.assertEqual(maximum, 27.83)

    @patch("csv.writer")
    @patch("builtins.open", new_callable=mock_open)
    def test_write_temperatures_to_csv(self, mock_file, mock_csv_writer):
        # Test that temperatures are correctly written to a CSV
        temperatures_celsius = [25.83, 27.33]
        write_temperatures_to_csv(temperatures_celsius, "fake_output.csv")
        mock_file.assert_called_once_with("fake_output.csv", 'w', newline='')
        mock_csv_writer.return_value.writerow.assert_any_call(['Temperature (C)'])
        mock_csv_writer.return_value.writerow.assert_any_call([25.83])
        mock_csv_writer.return_value.writerow.assert_any_call([27.33])

class IntegrationTestCase(unittest.TestCase):

    @patch("__main__.write_temperatures_to_csv")
    @patch("__main__.read_temperatures_from_csv")
    def test_process_temperature_data(self, mock_read, mock_write):
        # Integration test to ensure that the whole process works as expected
        mock_read.return_value = [25.83, 27.33, 24.39, 27.83]
        process_temperature_data("fake_input.csv", "fake_output.csv")
        mock_read.assert_called_once_with("fake_input.csv")
        mock_write.assert_called_once()


unittest.main(argv=[''], exit=False)


....
----------------------------------------------------------------------
Ran 4 tests in 0.025s

OK


Statistics:
Average: 26.34°C
Minimum: 24.39°C
Maximum: 27.83°C
